# 导入需要的包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/root/miniconda3/envs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 读取数据

In [2]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [ ]:
ds[:3]
'''
数据集构成
output：标准回答
input：instruction的补充内容（可以为空）
instruction：问题
'''

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./Langboat")
tokenizer
'''
tokenizer参数解释
padding_side='left'：表示padding添加在左侧
truncation_side='right'：表示当超过长度时在右侧进行截断
special_tokens：额外添加的特殊token
'''

BloomTokenizerFast(name_or_path='./Langboat', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

## 数据处理函数

In [ ]:
# 传入参数为数据集ds
def process_func(example):
    MAX_LENGTH = 256  # 设置tokens的最大长度
    input_ids, attention_mask, labels = [], [], []  # 创建返回对象
    # 将instruction和input统一作为输入
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)  # 添加结束标志
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 对于生成式任务，计算loss时只和输出（response/output）计算，所以将输入部分的labels设置为-100
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]  
    # 超出长度进行截断
    if len(input_ids) > MAX_LENGTH: 
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
# 将ds转为tokens
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [ ]:
# 检查数据集处理效果
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [ ]:
# 检查labels设置情况
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

# 构建模型

In [11]:
model = AutoModelForCausalLM.from_pretrained("./Langboat")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#print(model.device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(42437, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (l

# 设置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=8, # 每8个batch累积一次梯度，然后再进行一次优化更新
    logging_steps=50,
    num_train_epochs=2
)

# 搭建训练器&训练

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True) # 批处理器，按照tokenizer的标准对数据进行截断和补全
)


/tmp/ipykernel_1137/1470884895.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Step,Training Loss
50,3.118400
100,2.865600
150,2.872900
200,2.829500
250,2.757700
300,2.690400
350,2.651800
400,2.657900
450,2.574400
500,2.593900


TrainOutput(global_step=1678, training_loss=2.0937835752466722, metrics={'train_runtime': 1128.3875, 'train_samples_per_second': 47.604, 'train_steps_per_second': 1.487, 'total_flos': 1.1646725667618816e+16, 'train_loss': 2.0937835752466722, 'epoch': 1.9995532390171258})

# 推理验证

In [15]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [17]:
ipt = "Human: {}\n{}".format("如何获得导师的青睐？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

[{'generated_text': 'Human: 如何获得导师的青睐？\n\nAssistant: 获得导师的青睐是一种成功的重要途径。首先，你应该有足够的能力、经验和技能来获得导师的青睐。其次，你应该清楚地表达你的个性，并提供一些相关的例子来证明你的能力。在找到一位优秀的导师后，你应该表现出对你的兴趣和热情，与导师保持良好的沟通，帮助他们在你身上找到优势互补之处。此外，建立良好的关系和建立信任也是非常重要的。在成功获得导师的青睐”后，你应该感激他们为你带来的帮助和支持。'}]

# 总结

trainer 接收的数据格式固定，需要注意

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

注意在gpu上进行训练！！！